In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import ConnectionPatch

# Obtendo os dados do Power BI
categoria = dataset['LINHA_ABREV']
valores = dataset['Vendas Total %']

# Definindo o limiar percentual que determina quais informações entram na fatia de "Outros"
threshold = 0.05 #* 100 

# Criando um DataFrame para facilitar a manipulação
df = pd.DataFrame({
    'categoria': categoria,
    'valores': valores
})

# Agrupando as categorias que têm valor menor que o limiar em 'Outros'
df_sum = df.groupby('categoria')['valores'].sum().reset_index()
df_sum = df_sum.sort_values(by='valores', ascending=False)
df_above_threshold = df_sum[df_sum['valores'] >= threshold].copy()
df_below_threshold = df_sum[df_sum['valores'] < threshold].copy()

# Somando as categorias abaixo do limiar em uma categoria "Outros" e concatenando com o dataframe principal (df_above_threshold)
if not df_below_threshold.empty:
    other_sum = df_below_threshold['valores'].sum()
    new_row = pd.DataFrame({'categoria': ['Outros'], 'valores': [other_sum]})
    df_above_threshold = pd.concat([df_above_threshold, new_row], ignore_index=True)

# Atualizando os dados para o gráfico
categoria = df_above_threshold['categoria']
valores = df_above_threshold['valores']

# Criando a figura e os eixos customizáveis para os gráficos
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
fig.subplots_adjust(left=0.1, right=1, top=1, bottom=0)  # Ajuste manual de preenchimento de espaço da figura

# Valores para configuração manual da posição de cada gráfico individualmente
ax1.set_position([0.02, 0.1, 0.6, 0.82])  # [left, bottom, width, height]
ax2.set_position([0.59, 0.2, 0.3, 0.68])  # [left, bottom, width, height]

# Definindo as cores das categorias do gráfico de rosca
cores = ['#1d0021','#2f0629', '#420d32', '#56143b', '#651a42', '#79214b', '#8c2854'] 
if len(categoria) > len(cores):
    cores = cores * (len(categoria) // len(cores) + 1)

# Definindo parâmetros do gráfico de rosca e o plotando
wedges, texts = ax1.pie(
    valores,
    startangle=0,
    counterclock=False,
    colors=cores[:len(categoria)],
    wedgeprops=dict(width=0.5),
    labels=None
)

# Criando um círculo branco no centro para transformar em gráfico de rosca
my_circle = plt.Circle((0, 0), 0.7, color='white')
ax1.add_artist(my_circle)

# Garantir que o gráfico seja circular
ax1.axis('equal')

# Configurar as propriedades do rótulo e seta que liga o rótulo à fatia
bbox_props = dict(boxstyle="square,pad=0.2", fc="w", ec="k", lw=0.72)
kw = dict(arrowprops=dict(arrowstyle="-"), zorder=0, va="center")

for i, p in enumerate(wedges):
    ang = (p.theta2 - p.theta1) / 2. + p.theta1
    y = np.sin(np.deg2rad(ang))
    x = np.cos(np.deg2rad(ang))
    percentual = valores[i] * 100 # Multiplicação dos valores que estão em escala percentual na base utilizada
    horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
    connectionstyle = f"angle,angleA=0,angleB={ang}"
    kw["arrowprops"].update({"connectionstyle": connectionstyle})
    
    # Configurando o rótulo baseado na posição para cada categoria. x e y são as coordenadas do rótulo, fontsize determina o tamanho da fonte e alterando o
    # valor de {percentual:.2f} conseguimos alterar o número de casas decimais. É possível adicionar ou remover categorias manualmente.
    if categoria[i] == "ELETROFISIOLOGIA":
        ax1.annotate(f"{categoria[i]} ({percentual:.2f}%)", xy=(x, y), xytext=(.5 * np.sign(x), 1.2 * y),
                    horizontalalignment=horizontalalignment, fontsize=15, **kw)
    elif categoria[i] == "RADIOLOGIA":
        ax1.annotate(f"{categoria[i]} ({percentual:.2f}%)", xy=(x,y), xytext=(0.5 * np.sign(x), 1.2 * y),
                    horizontalalignment=horizontalalignment, fontsize=15, **kw)
    elif categoria[i] == "CRM":
        ax1.annotate(f"{categoria[i]} ({percentual:.2f}%)", xy=(x,y), xytext=(1 * np.sign(x), 0.2 * y),
                    horizontalalignment=horizontalalignment, fontsize=15, **kw)
    elif categoria[i] == "Outros":
        ax1.annotate(f"{categoria[i]} ({percentual:.2f}%)", xy=(x,y), xytext=(1.2 * np.sign(x), 1.2 * y),
                    horizontalalignment=horizontalalignment, fontsize=15, **kw)
    elif categoria[i] == "VASCULAR":
        ax1.annotate(f"{categoria[i]} ({percentual:.2f}%)", xy=(x,y), xytext=(1.15 * np.sign(x), 1.2 * y),
                    horizontalalignment=horizontalalignment, fontsize=15, **kw)
    elif categoria[i] == "NUTRIÇÃO":
        ax1.annotate(f"{categoria[i]} ({percentual:.2f}%)", xy=(x,y), xytext=(0.6 * np.sign(x), 1.2 * y),
                    horizontalalignment=horizontalalignment, fontsize=15, **kw)
    else:
        # Mantém a posição padrão para outras categorias
        ax1.annotate(f"{categoria[i]} ({percentual:.2f}%)", xy=(x, y), xytext=(1.0 * np.sign(x), 1.2 * y),
                    horizontalalignment=horizontalalignment, fontsize=15, **kw)

# Se a fatia "Outros" existe, plotar o gráfico de barras detalhado
if 'Outros' in df_above_threshold['categoria'].values:
    outros_data = df_below_threshold.copy()
    outros_data['valores'] = outros_data['valores'] * 100 # Multiplicação por 100 para converter valores em percentual da nossa base
    
    # Plotando o gráfico de barras
    bottom = 100 # Posição inicial da barra
    width = .2

    for j, (height, label) in enumerate(reversed([*zip(outros_data['valores'], outros_data['categoria'])])):
        bottom -= height # Atualiza a posição da barra
        bc = ax2.bar(0, height, width, bottom=bottom, label=label, color='gray', alpha=0.1 + 0.25 * j) # parâmetros do gráfico de barra
            # Coordenadas padrão
        xy = (0, bottom + height / 2)
        xytext = (0.5, 0)  
            # Ajuste de coordenadas específico para cada categoria
        if label == "A.T.":
            xytext = (40, 2)  
        elif label == "ESTRUTURAL":
            xytext = (40, 2) 
        elif label == "ENDOSCOPIA":
            xytext = (40, 2)  
        elif label == "TER. ABLATIVAS":
            xytext = (36, 2)  

        ax2.annotate(f"{label} ({height:.2f}%)", # Rótulo da barra, é possível alterar o número de casas decimais nessa linha
                xy=xy,  # Ponto central da barra
                xytext=xytext,
                textcoords="offset points",
                ha='left', va='center', fontsize=15, color='black',
                arrowprops=dict(arrowstyle="-", connectionstyle="arc3,rad=0"))


    ax2.set_title('Outros:', fontsize=15) # definição do título do gráfico de barras
    #ax2.legend() é possível ativar as legendas no gráfico de barras nesta linha
    ax2.axis('off')
    ax2.set_xlim(- 2.5 * width, 2.5 * width)

    # Conectar os gráficos usando ConnectionPatch ########################## Corrigir esta parte do código
 #   outros_index = df_above_threshold.index[df_above_threshold['categoria'] == 'Outros'][0]
 #   theta1, theta2 = wedges[outros_index].theta1, wedges[outros_index].theta2
 #   theta_medio = (theta2 + theta1) / 2  # Calcule o ângulo médio
 #   center, r = wedges[outros_index].center, wedges[outros_index].r
 #   bar_height = sum(outros_data['valores'])*100

    # Linha superior
#    x = r * np.cos(np.pi / 180 * theta_medio) + center[0]
#    y = r * np.sin(np.pi / 180 * theta_medio) + center[1]
#    con = ConnectionPatch(xyA=(0, bar_height), coordsA=ax2.transData,
#                        xyB=(x, y), coordsB=ax1.transData)
#    con.set_color([0, 0, 0])
#    con.set_linewidth(1)
#    ax2.add_artist(con)


    # Linha inferior
#    x = r * np.cos(np.pi / 180 * theta_medio) + center[0]
#    y = r * np.sin(np.pi / 180 * theta_medio) + center[1]
#    con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax2.transData,
#                          xyB=(x, y), coordsB=ax1.transData)
#    con.set_color([0, 0, 0])
#    con.set_linewidth(1)
#    ax2.add_artist(con)

plt.show()
